In [1]:
from jax import random, numpy as jnp
import learning_rules

In [2]:



kernel = jnp.array([[0.3,0.3], [0.3,0.3], [0.3,0.3]], dtype=jnp.float32)
# Initialize parameters
eligibility_params = {"ALIFCell_0":{'thr': 0.6, 'gamma': 1,'alpha': 0.2865048, 'rho': 0.2865048,'betas': jnp.array([0,1.7,1.7])},
                       
                       'ReadOut_0':{'kappa': 0.2865048,'feedback_weights':kernel}
}




# eligibility inputs
v_init = jnp.array([[[0.6, 0.6, 0.6], [1, 2, 3]],
                    [[0.771, 0.771, 0.771], [1, 2, 3]],
                    [[0.821, 0.821, 0.821], [1, 2, 3]],
                    [[0.23, 1.13, 1.13], [1, 2, 3]]],dtype=jnp.float32) #(n_t, n_batch, n_rec,)
A_thr_init = jnp.array([[[0.6,0.6,0.6], [2,3,4]],
                        [[0.6,0.6,0.6], [2,3,4]],
                        [[0.6,2.3,2.3], [2,3,4]],
                        [[0.6,1.08,1.08], [2,3,4]]], dtype=jnp.float32) #(n_t, n_batch, n_rec)

x = jnp.array([[[1,1],[0,1]],
               [[1,1],[0,1]],
               [[1,1],[0,1]],
               [[1,1],[0,1]]], dtype=jnp.float32) # (n_t, n_batch, n_in) simulating here the inputs coming from input neurons 


r = jnp.array([[[0,0,0],[0,0,0]],
               [[0,0,0],[0,0,0]],
               [[0,0,0],[0,0,0]],
               [[0,0,0],[0,0,0]]], dtype=jnp.float32)

z = jnp.array([[[0,0,0],[0,1,1]],
               [[1,1,1],[0,1,0]],
               [[1,0,0],[0,1,1]],
               [[0,1,1],[0,1,1]]], dtype=jnp.float32)



eligibility_input = v_init, A_thr_init, r, x

# Outputs
true_y = jnp.array([[[2,3], [1,2]],
                    [[2,3], [1,2]],
                    [[2,3], [1,2]],
                    [[2,3], [1,2]]], dtype=jnp.float32) # (n_t, n_batch, n_out)
y = jnp.array([[[1,0], [2,2]],
               [[1,0], [2,2]],
               [[1,0], [2,2]],
               [[1,0], [2,2]]], dtype=jnp.float32)

true_y = jnp.transpose(true_y, (1,0,2)) # (n_b, n_t, n_out)
y = jnp.transpose(y, (1,0,2)) # (n_b, n_t, n_out)
# inputs
batch_inputs = (y, true_y, eligibility_input)

#eligibility_carries
v_eligibility_vector = jnp.array([[0,0], [0,0]], dtype=jnp.float32) #(n_batch, n_in)
a_eligibility_vector = jnp.array([[[0,0],[0,0],[0, 0]], [[0,0],[0,0],[0, 0]]], dtype=jnp.float32) # (n_batch, n_rec,n_in)
low_pass_trace = jnp.array([[[0,0],[0,0],[0, 0]], [[0,0],[0,0],[0, 0]]], dtype=jnp.float32) # (n_batch, n_rec,n_in)
psi = jnp.array([[0.,0.,0.],[0.,0.,0.]],dtype=jnp.float32) # (n_batch, n_in)
init_eligibility_carries_in = {"v_eligibility_vector":v_eligibility_vector,
                                        "a_eligibility_vector":a_eligibility_vector,
                                        "psi": psi,
                                        "low_pass_eligibility_trace":low_pass_trace
                                        }

c_reg =10
f_target = 10
LS_avail =1
trial_lenght = jnp.array([4,4])

In [3]:
print(z.shape)

(4, 2, 3)


In [4]:
vectorized_update = learning_rules.e_prop_vectorized(batch_init_eligibility_carries=init_eligibility_carries_in,
                                                          batch_inputs=batch_inputs, 
                                                          eligibility_params=eligibility_params,
                                                           LS_avail= LS_avail,
                                                           z = z, trial_length=trial_lenght,
                                                           f_target=f_target, c_reg=c_reg
                                                           )


In [5]:
vectorized_update

Array([[-5.147802, -5.454222, -5.454222],
       [-5.147802, -5.454222, -5.454222]], dtype=float32)

In [6]:
online_update = learning_rules.e_prop_online(batch_init_eligibility_carries=init_eligibility_carries_in,
                                                          batch_inputs=batch_inputs, 
                                                          eligibility_params=eligibility_params,
                                                           LS_avail= LS_avail,
                                                           z = z, trial_length=trial_lenght,
                                                           f_target=f_target, c_reg=c_reg)

In [7]:
online_update

Array([[-5.1478024, -5.454222 , -5.454222 ],
       [-5.1478024, -5.454222 , -5.454222 ]], dtype=float32)

### Testing updates with CA

If we have diffusion kernels that are zeroes, the diffusion updates is the same as e-prop

In [8]:
radius = 1
k = jnp.zeros((1,1,2*radius+1,2*radius+1)) # 1 output channels, 1 input channel, height 3, width 3

diff_params = radius, k

initial_error_grid = jnp.zeros((2, 1, 10, 10))

neurons_loc = jnp.array([[0,0], [0,1], [1,0]])
print(neurons_loc.shape)

(3, 2)


In [9]:
params = (eligibility_params, diff_params)
batch_init_carries = (init_eligibility_carries_in, initial_error_grid)

In [10]:
diff_updates = learning_rules.neuromod_online(batch_init_carries=batch_init_carries
                                              ,cells_loc=neurons_loc, params=params, 
                                              batch_inputs= batch_inputs, 
                                             LS_avail= LS_avail, z=z, 
                                             trial_length=trial_lenght, f_target=f_target, c_reg=c_reg)

task_error_shape[[-1. -3.]
 [ 1.  0.]]
task_error_shape[[-1. -3.]
 [ 1.  0.]]
task_error_shape[[-1. -3.]
 [ 1.  0.]]
task_error_shape[[-1. -3.]
 [ 1.  0.]]


In [11]:
diff_updates

Array([[-5.1478024, -5.454222 , -5.454222 ],
       [-5.1478024, -5.454222 , -5.454222 ]], dtype=float32)

In [12]:
radius = (jnp.shape(k)[2] -1) // 2

In [16]:
jnp.array(radius, dtype=jnp.int32)

Array(1, dtype=int32)